In [1]:
import os

In [2]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

In [3]:
connection_string ="DefaultEndpointsProtocol=https;AccountName=azlndstorage;AccountKey=cR28uFLHRAYRuV9srs7+TAiTPitoB8GiQmH5AwCPIdiUONblm1gnU7hDMocnmCDZPv88FAAnyCpwSRF9IL2Zuw==;EndpointSuffix=core.windows.net"

In [4]:
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

In [5]:
container_name ='lndfiles'


In [82]:
# container_client = blob_service_client.create_container(container_name)

In [159]:
os.listdir('./docs')

['cont4.PDF']

In [160]:
 for i in os.listdir('./docs'):
    blob_client = blob_service_client.get_blob_client(
    container=container_name,
    blob = i
    )
    
    with open(os.path.join('.', 'docs', i), 'rb') as data:
        blob_client.upload_blob(data)

In [38]:
# blob_list = container_client.list_blobs()
# for blob in blob_list:
#     print(blob.name)

In [6]:
import requests
import json

In [7]:
service_name ="azlndcogsearch"
api_version = "2019-05-06"

headers = {
    'Content-Type' : 'application/json',
    'api-key':"34B9F063F1A407EEFEF10E58181C5261"
}

In [8]:
datasource_name = "azlnd"
uri = f"https://azlndcogsearch.search.windows.net/datasources?api-version=2019-05-06"

body = {
    "name": datasource_name,
    "type": "azureblob",
  "credentials": {"connectionString": connection_string},
    "container": {"name": container_name}
}



In [9]:
resp = requests.post(uri, headers=headers, data=json.dumps(body))
print(resp.status_code)
print(resp.ok)

201
True


In [22]:
null =""

In [10]:
 skillset = {
  "description": "Extract text from images and merge with content text to produce merged_text. Also extract key phrases from pages",
  "skills":
  [
    {
      "description": "Extract text (plain and structured) from image.",
      "@odata.type": "#Microsoft.Skills.Vision.OcrSkill",
      "context": "/document/normalized_images/*",
      "defaultLanguageCode": "en",
      "detectOrientation": True,
      "inputs": [
        {
          "name": "image",
          "source": "/document/normalized_images/*"
        }
      ],
      "outputs": [
        {
          "name": "text"
        }
      ]
    },
    {
      "@odata.type": "#Microsoft.Skills.Text.MergeSkill",
      "description": "Create merged_text, which includes all the textual representation of each image inserted at the right location in the content field.",
      "context": "/document",
      "insertPreTag": " ",
      "insertPostTag": " ",
      "inputs": [
        {
          "name":"text", "source": "/document/content"
        },
        {
          "name": "itemsToInsert", "source": "/document/normalized_images/*/text"
        },
        {
          "name":"offsets", "source": "/document/normalized_images/*/contentOffset"
        }
      ],
      "outputs": [
        {
          "name": "mergedText", "targetName" : "merged_text"
        }
      ]
    },
    {
      "@odata.type": "#Microsoft.Skills.Text.LanguageDetectionSkill",
      "inputs": [
        { "name": "text", "source": "/document/content" }
      ],
      "outputs": [
        { "name": "languageCode", "targetName": "languageCode" }
      ]
    },
    {
      "@odata.type": "#Microsoft.Skills.Text.SplitSkill",
      "textSplitMode" : "pages",
      "maximumPageLength": 4000,
      "inputs": [
        { "name": "text", "source": "/document/content" },
        { "name": "languageCode", "source": "/document/languageCode" }
      ],
      "outputs": [
        { "name": "textItems", "targetName": "pages" }
      ]
    },
    {
      "@odata.type": "#Microsoft.Skills.Text.KeyPhraseExtractionSkill",
      "context": "/document/pages/*",
      "inputs": [
        { "name": "text", "source": "/document/pages/*" },
        { "name":"languageCode", "source": "/document/languageCode" }
      ],
      "outputs": [
        { "name": "keyPhrases", "targetName": "keyPhrases" }
      ]
    }],
  "cognitiveServices": {
    "@odata.type": "#Microsoft.Azure.Search.CognitiveServicesByKey",
    "description": "sterlinglegal cogservices",
    "key": "773626f8e39a43c9bca9517d59a9c58b"
  }
}

In [11]:
skillset_name = 'azlndskillset'
uri = f"https://azlndcogsearch.search.windows.net/skillsets/{skillset_name}?api-version=2019-05-06"



In [12]:
resp = requests.put(uri, headers=headers, data=json.dumps(skillset))
print(resp.status_code)
print(resp.ok)

201
True


In [13]:
index = {
  "fields": [
    {
      "name": "id",
      "type": "Edm.String",
      "key": True,
      "searchable": False,
      "filterable": False,
      "facetable": False,
      "sortable": True
    },
    {
      "name": "metadata_storage_name",
      "type": "Edm.String",
      "searchable": True,
      "filterable": False,
      "facetable": False,
      "sortable": True
    },
    {
      "name": "content",
      "type": "Edm.String",
      "searchable": True,
      "filterable": False,
      "facetable": False,
      "sortable": False
    },
    {
      "name": "languageCode",
      "type": "Edm.String",
      "searchable": True,
      "filterable": False,
      "facetable": False,
      "sortable": False
    },
    {
      "name": "keyPhrases",
      "type": "Collection(Edm.String)",
      "searchable": True,
      "filterable": False,
      "facetable": False,
      "sortable": False
    },
        {
      "name": "people",
      "type": "Collection(Edm.String)",
      "searchable": True,
      "filterable": False,
      "facetable": False,
      "sortable": False
    },
      
        {
      "name": "organizations",
      "type": "Collection(Edm.String)",
      "searchable": True,
      "filterable": False,
      "facetable": False,
      "sortable": False
    },
        {
      "name": "locations",
      "type": "Collection(Edm.String)",
      "searchable": True,
      "filterable": False,
      "facetable": False,
      "sortable": False
    },
        {
      "name": "entities",
      "type": "Collection(Edm.String)",
      "searchable": True,
      "filterable": False,
      "facetable": False,
      "sortable": False
    },
         
       
    {
      "name": "pii_entities",
      "type": "Collection(Edm.String)",
      "searchable": True,
      "filterable": False,
      "facetable": False,
      "sortable": False
    },
    {
      "name": "merged_text",
      "type": "Edm.String",
      "searchable": True,
      "filterable": False,
      "facetable": False,
      "sortable": False
    }
  ]
}

In [14]:
index_name = 'azlndindex'
uri = f"https://{service_name}.search.windows.net/indexes/{index_name}?api-version={api_version}"



In [15]:
resp = requests.put(uri, headers=headers, data=json.dumps(index))
print(resp.status_code)
print(resp.ok)

201
True


In [16]:
indexer_name = "azlndindexer"

indexer = {
  "name": indexer_name,
  "dataSourceName" : datasource_name,
  "targetIndexName" : index_name,
  "skillsetName" : skillset_name,
  "fieldMappings" : [
    {
      "sourceFieldName" : "metadata_storage_path",
      "targetFieldName" : "id",
      "mappingFunction" : {"name": "base64Encode"}
    },
    {
      "sourceFieldName" : "metadata_storage_name",
      "targetFieldName" : "metadata_storage_name",
    },
    {
      "sourceFieldName" : "content",
      "targetFieldName" : "content"
    }
  ],
  "outputFieldMappings" :
  [
    {
      "sourceFieldName" : "/document/merged_text",
      "targetFieldName" : "merged_text"
    },
    {
      "sourceFieldName" : "/document/pages/*/keyPhrases/*",
      "targetFieldName" : "keyPhrases"
    },
    {
      "sourceFieldName" : "/document/pages/*/people/*",
      "targetFieldName" : "people"
    },
    {
      "sourceFieldName" : "/document/pages/*/organizations/*",
      "targetFieldName" : "organizations"
    },
     {
      "sourceFieldName" : "/document/pages/*/locations/*",
      "targetFieldName" : "locations"
    },
     {
      "sourceFieldName" : "/document/pages/*/entities/*",
      "targetFieldName" : "entities"
    },
      {
      "sourceFieldName" : "/document/pages/*/pii_entities/*",
      "targetFieldName" : "pii_entities"
    },
    {
      "sourceFieldName": "/document/languageCode",
      "targetFieldName": "languageCode"
    }
  ],
  "parameters":
  {
    "maxFailedItems": -1,
    "maxFailedItemsPerBatch": -1,
    "configuration":
    {
      "dataToExtract": "contentAndMetadata",
      "parsingMode": "default",
      "firstLineContainsHeaders": False,
      "delimitedTextDelimiter": ",",
      "imageAction": "generateNormalizedImages"
    }
  }
}

In [17]:

uri = f"https://{service_name}.search.windows.net/indexers/{indexer_name}?api-version={api_version}"



In [18]:
resp = requests.put(uri, headers=headers, data=json.dumps(indexer))
print(resp.status_code)
print(resp.ok)

201
True


In [19]:
uri = f"https://{service_name}.search.windows.net/indexers/{indexer_name}/status?api-version={api_version}"

resp = requests.get(uri, headers=headers)
print(resp.status_code)
print(resp.json().get('lastResult').get('status'))
print(resp.ok)

200
inProgress
True


In [20]:
# Base URL
url = 'https://{}.search.windows.net/indexes/{}/docs'.format(service_name, index_name)
# API version is required
url += '?api-version={}'.format(api_version)
# Search query of "COMPLIANCE"
url += '&search= our history'  #Searching for key words here
# Return the count of the results
url += '&$count=true'
print(url)

resp = requests.get(url, headers=headers)
print(resp.status_code)

https://azlndcogsearch.search.windows.net/indexes/azlndindex/docs?api-version=2019-05-06&search= our history&$count=true
200


In [21]:
search_results = resp.json()


In [22]:
search_results.keys()

dict_keys(['@odata.context', '@odata.count', 'value'])

In [23]:
search_results['@odata.context']

"https://azlndcogsearch.search.windows.net/indexes('azlndindex')/$metadata#docs(*)"

In [24]:
search_results['@odata.count']

1

In [25]:
len(search_results['value'])

1

In [26]:
search_results['value'][0].keys()

dict_keys(['@search.score', 'id', 'metadata_storage_name', 'content', 'languageCode', 'keyPhrases', 'people', 'organizations', 'locations', 'entities', 'pii_entities', 'merged_text'])

In [27]:
for result in search_results['value']:
    print('doc Name: {}, Search Score {}'.format(result['metadata_storage_name'], result['@search.score']))

doc Name: Sterling-Bank-Staff-Handbook-2015_47803.pdf, Search Score 2.046639


In [30]:
print(search_results['value'][0]['merged_text'][:])


1 

 

 

 

 

 

 

 

 

                            

 

    

 

 

 

 

 

 

 

[image: image0.png] Sterling Bank 

[image: image1.jpg] @IPBRF employment do drug disciplinary timekeepingupdated completed voluntarily resignationgrievanceagreebenefitsactionsform employee written requirement POIICI employer @123RF handbook: retirement HeIS induction To Kel communicatedresultprocessrevised sexual informationstate demotion termination supervisors proof company & stemming additional statement #thholding@private harassment enrollment dismissa/ designatedarearules. curent required settlements business vacationdiscrimination terms employee speled terminate Concurrently procedures 01123 PIPER 



2 

 

 

Table of Contents 
 

Chapter 1 – About the Bank .................................................................................................... 3 

 

Chapter 2 – Human Resource Policy  ................................................................................... 10 

 

C

In [24]:
url = 'https://{}.search.windows.net/indexes/{}/docs'.format(service_name, index_name)
url += '?api-version={}'.format(api_version)
url += '&search="digital risk management framework"'
url += '&$count=true'
print(url)

resp = requests.get(url, headers=headers)
print(resp.status_code)

https://az-erm-cogsearch-prod.search.windows.net/indexes/ermindex/docs?api-version=2019-05-06&search="digital risk management framework"&$count=true
200


In [25]:
search_results = resp.json()
search_results['@odata.count']

2

In [26]:
for result in search_results['value']:
    print('PDF Name: {}, Search Score {}'.format(result['metadata_storage_name'], result['@search.score']))

PDF Name: Digital Risk Mangement Framework .pdf, Search Score 10.578436
PDF Name: DIGITAL RISK MANAGEMENT KSS.pptx, Search Score 5.7255125


In [25]:
print(search_results['value'][1]['merged_content'][:])


  

   1 of 119  

    Internal use     
 

 

Policy | INFORMATION SECURITY 
  
 SBP/RMG/ISO/19/0001 
 Version 2.0 
 

 

 

 

 

 

 

 

   

 

Document Control Sheet 

Version and Update History 

Date Document Version Document Revision History Document Author/Reviser 

July 2018 2.0 Document creation  Risk Management  

September 2018  2.0 Document Review   EXCO and BRMC 

November 2018 2.0 Document Approval   Board of Directors 

 
Change Control 

Change Clause/Frequency 

The contents of this document are subject to change control on a twelve (12) months review cycle. 

 

 

 

 

Policy Number SBP/RMG/ISO/19/0001      

Policy Owner  Risk Management Group 

Policy Status Approved 

Date Signed Off  November 2018 

Review Frequency  12 Months 

Next Review Date   October 2019 

[image: image0.png][image: image1.png][image: image2.png][image: image3.png]  



  

2 of 119  

    Internal use     

Table of Contents 

INFORMATION SECURITY POLICY FRAMEWORK      7 

SECTION I: 

In [72]:
# print(search_results['value'][1]['merged_text'][:600])

In [61]:
# url = 'https://{}.search.windows.net/indexes/{}/docs'.format(service_name, index_name)
# url += '?api-version={}'.format(api_version)
# url += '&search=*'
# url += '&$count=true'

# resp = requests.get(url, headers=headers)
# search_results = resp.json()

# for result in search_results['value']:
#     print(result['metadata_storage_name'])
#     print(result['keyPhrases'])

In [27]:
term = ("risk control and mitigation")
url = 'https://{}.search.windows.net/indexes/{}/docs'.format(service_name, index_name)
url += '?api-version={}'.format(api_version)
url += '&search="{}"'.format(term)
url += '&$count=true'
# url += '&$top=1'
print(url)

resp = requests.get(url, headers=headers)
print(resp.status_code)

search_results = resp.json()

print("Results Found: {}, Results Returned: {}".format(search_results['@odata.count'], len(search_results['value'])))
print("Highest Search Score: {}".format(search_results['value'][0]['@search.score']))

https://az-erm-cogsearch-prod.search.windows.net/indexes/ermindex/docs?api-version=2019-05-06&search="risk control and mitigation"&$count=true
200
Results Found: 2, Results Returned: 2
Highest Search Score: 4.857489


In [39]:
# url = 'https://{}.search.windows.net/indexes/{}/docs'.format(service_name, index_name)
# url += '?api-version={}'.format(api_version)
# url += '&search=contract'
# url += '&$count=true'
# url += '&$top=2'
# url += '&$skip=2'
# print(url)

# resp = requests.get(url, headers=headers)
# print(resp.status_code)

# search_results = resp.json()

# print("Results Found: {}, Results Returned: {}".format(search_results['@odata.count'], len(search_results['value'])))
# print("Highest Search Score: {}".format(search_results['value'][0]['@search.score']))

In [37]:
url = 'https://{}.search.windows.net/indexes/{}/docs'.format(service_name, index_name)
url += '?api-version={}'.format(api_version)
url += '&search=Sustainability Principles'
url += '&$count=true'
url += '&highlight=merged_text'

resp = requests.get(url, headers=headers)
print(resp.status_code)

search_results = resp.json()

200


In [43]:
# search_results['value'][1]['@search.highlights']['merged_text']

In [38]:
from IPython.display import display, HTML

for highlight in search_results['value'][0]['@search.highlights']['merged_text']:
    display(HTML(highlight))

In [40]:
# service_name ="az-erm-cogsearch-prod"
# api_version = "2019-05-06"

# headers = {
#     'Content-Type' : 'application/json',
#     'api-key':"B41FFAF7BD26E5416FFD3058626C4D5C"
# }



# index_name = 'ermindex'
# uri = f"https://{service_name}.search.windows.net/indexes/{index_name}?api-version={api_version}"


import urllib
search_term1 = 'Sustainability Principles '
# search_term2 = 'Risk identification process '
# search_term3 = 'New Product Risk Assessment (NPRA) '
url = 'https://{}.search.windows.net/indexes/{}/docs'.format(service_name, index_name)  #serv
url += '?api-version={}'.format(api_version)
url += '&search="{}"'.format(search_term1 ) #this is the searched key word " digital risk"
url += '&$count=true'
# url += '&$top=1'
url += '&highlight=merged_text'
url += '&highlightPreTag=' + urllib.parse.quote('<span style="background-color: #a3f71d">', safe='')
url += '&highlightPostTag=' + urllib.parse.quote('</span>', safe='')

resp = requests.get(url, headers=headers)
print(url)
print(resp.status_code)

search_results = resp.json()

https://azlndcogsearch.search.windows.net/indexes/azlndindex/docs?api-version=2019-05-06&search="Sustainability Principles "&$count=true&highlight=merged_text&highlightPreTag=%3Cspan%20style%3D%22background-color%3A%20%23a3f71d%22%3E&highlightPostTag=%3C%2Fspan%3E
200


In [31]:
# resp.json()

In [42]:
for result in search_results['value']:
    display(HTML('<h4>' + str(result['metadata_storage_name']) + '</h4>'))

    for highlight in result['@search.highlights']['merged_text'][:]:
        display(HTML(highlight))
#         print(result['translated_text'][:])
    

In [42]:
b = (HTML('<h4>' + str(result['metadata_storage_name']) + '</h4>'))

In [43]:
b

In [44]:
c = HTML(highlight)

In [45]:
c

In [57]:
HTML(highlight)

In [30]:
import urllib

url = 'https://{}.search.windows.net/indexes/{}/docs'.format(service_name, index_name)
url += '?api-version={}'.format(api_version)
url += '&search="Contract acceptance"'
url += '&$count=true'
url += '&highlight=merged_text'
url += '&highlightPreTag=' + urllib.parse.quote('<span style="background-color: #f7f31d">', safe='')
url += '&highlightPostTag=' + urllib.parse.quote('</span>', safe='')

resp = requests.get(url, headers=headers)
print(url)
print(resp.status_code)

search_results = resp.json()

https://ikdocsearch.search.windows.net/indexes/5docsindex/docs?api-version=2019-05-06&search="Contract acceptance"&$count=true&highlight=merged_text&highlightPreTag=%3Cspan%20style%3D%22background-color%3A%20%23f7f31d%22%3E&highlightPostTag=%3C%2Fspan%3E
200


In [31]:
for result in search_results['value']:
    display(HTML('<h4>' + result['metadata_storage_name'] + '</h4>'))
    for highlight in result['@search.highlights']['merged_text']:
        display(HTML(highlight))

In [86]:
# for result in search_results['value']:
#     display(HTML('<h4>' + result['metadata_storage_name'] + '</h4>'))
#     for highlight in result['@search.highlights']['merged_text']:
#         display(HTML(highlight))